In [1]:
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
model = keras.models.load_model('model/efficientnetb0_27_0.818.h5', compile=False)

In [10]:
img = load_img('pink_potato.jpg', target_size=(224, 224))

In [11]:
X = np.array(img)
X = np.array([X])

preds = model.predict(X)

1/1 [==============================] - 0s 85ms/step


In [12]:
classes = [
    'Black Scurf',
 'Common Scab',
 'Dry Rot',
 'Blackleg',
 'Miscellaneous',
 'Healthy Potatoes',
 'Pink Rot'

]

dict(zip(classes, preds[0]))

{'Black Scurf': -5.3060904,
 'Common Scab': 0.46414873,
 'Dry Rot': -0.17142673,
 'Blackleg': -1.9026829,
 'Miscellaneous': -1.7460269,
 'Healthy Potatoes': 1.1571207,
 'Pink Rot': 6.5150166}

## Convert Keras to tflite

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model/potato-diseases-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\asus\AppData\Local\Temp\tmpm8kh72rp\assets


INFO:tensorflow:Assets written to: C:\Users\asus\AppData\Local\Temp\tmpm8kh72rp\assets


In [14]:
import tensorflow.lite as tflite

In [23]:
interpreter = tflite.Interpreter(model_path='model/potato-diseases-model.tflite')
interpreter.allocate_tensors()

In [35]:
input_details = interpreter.get_input_details()

In [31]:
input_index = interpreter.get_input_details()[0]['index']

In [36]:
# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

In [38]:
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [32]:
#need to see the output too
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 495,
  'shape': array([1, 7]),
  'shape_signature': array([-1,  7]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [33]:
output_index = interpreter.get_output_details()[0]['index']

In [34]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

ValueError: Cannot set tensor: Got value of type UINT8 but expected type FLOAT32 for input 0, name: serving_default_input_8:0 